# تنفيذ مشروع RADDet على Google Colab

هذا الدفتر يشرح خطوات تنفيذ مشروع RADDet بالكامل على بيئة Google Colab، بدءًا من تحميل المشروع والبيانات، وصولاً إلى تدريب النموذج وتقييمه.

### الخطوة الأولى: نسخ مستودع المشروع

أولاً، نقوم بنسخ مستودع المشروع من GitHub والانتقال إلى مجلد المشروع.

In [ ]:
!git clone https://github.com/ZhangAoCanada/RADDet.git
%cd RADDet

### الخطوة الثانية: تثبيت المكتبات المطلوبة

ثانياً، نقوم بتثبيت المكتبات المطلوبة للمشروع كما هو محدد في ملف `requirements.txt`.

In [ ]:
!pip install -r requirements.txt

### الخطوة الثالثة: تحميل مجموعة البيانات

ثالثاً، نقوم بتحميل مجموعة البيانات. الرابط الأصلي في المستودع لم يعد يعمل، لذلك سنستخدم الرابط المحدث من QUT Research Data Repository. حجم البيانات حوالي 556 ميجابايت.

In [ ]:
import os

# The official dataset is hosted on the QUT Research Data Repository.
# We will download the `RadDet-1T-128` version (~556 MB).

# We are inside /content/RADDet, so we save the zip file in the parent directory /content
zip_file_path = '../raddet-1t-128.zip'

if not os.path.exists(zip_file_path):
    print(f"Downloading the dataset...")
    !wget https://data.qut.edu.au/dataset/12a828ad-4289-4c56-9743-403c51547af3/resource/634905f1-2544-40e3-b878-9b9a4282bd04/download/raddet-1t-128.zip -O {zip_file_path}
else:
    print("Dataset already downloaded.")

### الخطوة الرابعة: فك ضغط البيانات

رابعاً، نقوم بفك ضغط ملف البيانات الذي تم تحميله. يتوقع المشروع أن تكون البيانات موجودة في مجلد `RadDet_data` داخل مجلد المشروع.

In [ ]:
import os

data_dir = 'RadDet_data/RadDet-1T-128'
zip_file_path = '../raddet-1t-128.zip'

if not os.path.exists(data_dir):
    print("Unzipping the data...")
    # The zip file is in the parent directory /content
    !unzip -q {zip_file_path} -d RadDet_data
else:
    print("Data already unzipped.")

### الخطوة الخامسة: استكشاف البيانات

خامساً، نستكشف البيانات عن طريق تحميل وعرض خريطة Range-Doppler. هذا يساعدنا على فهم طبيعة البيانات.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Path to a sample radar data file
file_path = 'RadDet_data/RadDet-1T-128/train/seq_000/r_d.npy'

if os.path.exists(file_path):
    # Load the radar data
    rd_data = np.load(file_path)

    # Display the radar image
    plt.figure(figsize=(8, 6))
    plt.imshow(rd_data, cmap='hot', interpolation='nearest')
    plt.title("Range-Doppler Map")
    plt.colorbar(label="Intensity")
    plt.xlabel("Doppler")
    plt.ylabel("Range")
    plt.show()
else:
    print(f"File not found at {file_path}. Please check the download and unzip steps.")

### الخطوة السادسة: تعديل الإعدادات للتدريب

سادساً، نقوم بتعديل ملف الإعدادات `RadDet_config.json` لتقليل وقت التدريب لأغراض العرض التوضيحي. سنقوم بتقليل عدد الـ `epochs` إلى 5.

In [ ]:
import json

config_path = 'RadDet_config.json'

# Read the config file
with open(config_path, 'r') as f:
    config = json.load(f)

# Modify the number of epochs for a quicker training demonstration
config['epochs'] = 5

# Write the modified config back
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)

print("Modified config file (epochs=5):")
!cat {config_path}

### الخطوة السابعة: تدريب النموذج

سابعاً، نبدأ عملية تدريب النموذج. قد تستغرق هذه العملية بعض الوقت حتى مع تقليل عدد الـ epochs.

In [ ]:
!python train.py --config_path RadDet_config.json

### الخطوة الثامنة: تقييم النموذج

ثامناً وأخيراً، بعد اكتمال التدريب، نقوم بتقييم أداء النموذج على مجموعة بيانات الاختبار.

In [ ]:
!python eval.py --config_path RadDet_config.json